<a href="https://colab.research.google.com/github/flaviopradoaquino/trabalhospark/blob/main/trabalhospark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Instalação de Pacotes**

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=c2c3b30cd7d4db18b7c9b8848adc156f238a14109cbd4d6d92e7cc3e515379df
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## **Importação dos Pacotes**

In [71]:
import string
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max, when, isnan
from pyspark.sql.functions import upper, col
from pyspark.sql import functions as sf
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

## **Inicia a sessão do Spark**

In [4]:
spark = SparkSession.builder.appName("AtividadeFinal").master('local[*]').getOrCreate()

## **Carrega os arquivos CSVs e adicona o encode para permitir acentuação**

In [5]:
idh = "/content/sample_data/idh_2010_tratado.csv"
enem = "/content/sample_data/enem_2022_tratado.csv"

In [6]:
df_i = spark.read.options(delimiter=";", header=True).option("encoding", "ISO-8859-1").csv(idh)
df_e = spark.read.options(delimiter=";", header=True).option("encoding", "ISO-8859-1").csv(enem)

## **Tratamento dos Dataframes**

In [7]:
df_i.show()

+------------+--------------------+-----+---+
|NM_FEDERACAO|        NM_MUNICIPIO| IDHM|IDH|
+------------+--------------------+-----+---+
|    Rondônia|ALTA FLORESTA D'O...|0.641|641|
|    Rondônia|           ARIQUEMES|0.702|702|
|    Rondônia|              CABIXI| 0.65|650|
|    Rondônia|              CACOAL|0.718|718|
|    Rondônia|          CEREJEIRAS|0.692|692|
|    Rondônia|   COLORADO DO OESTE|0.685|685|
|    Rondônia|          CORUMBIARA|0.613|613|
|    Rondônia|       COSTA MARQUES|0.611|611|
|    Rondônia|     ESPIGÃO D'OESTE|0.672|672|
|    Rondônia|       GUAJARÁ-MIRIM|0.657|657|
|    Rondônia|                JARU|0.689|689|
|    Rondônia|           JI-PARANÁ|0.714|714|
|    Rondônia|  MACHADINHO D'OESTE|0.596|596|
|    Rondônia|NOVA BRASILÂNDIA ...|0.643|643|
|    Rondônia| OURO PRETO DO OESTE|0.682|682|
|    Rondônia|       PIMENTA BUENO| 0.71|710|
|    Rondônia|         PORTO VELHO|0.736|736|
|    Rondônia|   PRESIDENTE MÉDICI|0.664|664|
|    Rondônia|          RIO CRESPO

In [8]:
### Exclusão de campos que não serão usados
df_i = df_i.drop(*('NM_FEDERACAO','IDHM'))

In [9]:
df_i.show()

+--------------------+---+
|        NM_MUNICIPIO|IDH|
+--------------------+---+
|ALTA FLORESTA D'O...|641|
|           ARIQUEMES|702|
|              CABIXI|650|
|              CACOAL|718|
|          CEREJEIRAS|692|
|   COLORADO DO OESTE|685|
|          CORUMBIARA|613|
|       COSTA MARQUES|611|
|     ESPIGÃO D'OESTE|672|
|       GUAJARÁ-MIRIM|657|
|                JARU|689|
|           JI-PARANÁ|714|
|  MACHADINHO D'OESTE|596|
|NOVA BRASILÂNDIA ...|643|
| OURO PRETO DO OESTE|682|
|       PIMENTA BUENO|710|
|         PORTO VELHO|736|
|   PRESIDENTE MÉDICI|664|
|          RIO CRESPO|643|
|      ROLIM DE MOURA|700|
+--------------------+---+
only showing top 20 rows



In [10]:
### Contagem de registros do Dataframe IDH antes de excluir os municípios duplicados
df_i.count()

5564

In [11]:
### Exclusão dos registros de municípios duplicados
df_i = df_i.dropDuplicates(['NM_MUNICIPIO'])

In [12]:
### Nova contagem dos registros de IDH
df_i.count()

5329

In [13]:
### Dataframe com dados do Enem
df_e.show()

+------------+-------+-----------+---------+------------+------------------+--------------------+-----------+----------+---------------+
|NU_INSCRICAO|TP_SEXO|TP_COR_RACA|TP_ESCOLA|IN_TREINEIRO|CO_MUNICIPIO_PROVA|  NO_MUNICIPIO_PROVA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+-------+-----------+---------+------------+------------------+--------------------+-----------+----------+---------------+
|    2,10E+11|      M|          2|        1|           0|           5300108|            Brasília|         DF|      NULL|           NULL|
|    2,10E+11|      M|          1|        1|           0|           5300108|            Brasília|         DF|      NULL|           NULL|
|    2,10E+11|      F|          2|        1|           0|           2925758|Presidente Tancre...|         BA|       565|            760|
|    2,10E+11|      M|          3|        1|           0|           3201308|           Cariacica|         ES|       416|            320|
|    2,10E+11|      M|          3|       

In [14]:
from pyspark.sql.functions import upper, col
from pyspark.sql import functions as sf

In [15]:
### Colocar emletras maiúsculas o campo 'Municípios' do Dataframe Enem
df_e = df_e.withColumn("NO_MUNICIPIO", upper(df_e.NO_MUNICIPIO_PROVA))

### Atriubuir um identificador único para cada registro do Dataframe Enem
df_e = df_e.select("*").withColumn("id", sf.monotonically_increasing_id())

### Exclusão dos campos que não serão usados
df_e = df_e.drop(*('NU_INSCRICAO', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'IN_TREINEIRO'))

In [16]:
df_e.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+
|      M|          2|        1|         DF|      NULL|           NULL|            BRASÍLIA|  0|
|      M|          1|        1|         DF|      NULL|           NULL|            BRASÍLIA|  1|
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2|
|      M|          3|        1|         ES|       416|            320|           CARIACICA|  3|
|      M|          3|        1|         PA|      NULL|           NULL|         SALINÓPOLIS|  4|
|      F|          3|        1|         RJ|      NULL|           NULL|      RIO DE JANEIRO|  5|
|      M|          2|        1|         PE|      NULL|           NULL|              RECIFE|  6|
|      F|          2|        1|         

In [17]:
### Join dos Datafrmaes IDH e Enem pelo Município
df = df_e.join(df_i, df_e.NO_MUNICIPIO == df_i.NM_MUNICIPIO, "left" )
df = df.drop('NM_MUNICIPIO')

In [18]:
df.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+----+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id| IDH|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+----+
|      M|          2|        1|         DF|      NULL|           NULL|            BRASÍLIA|  0| 824|
|      M|          1|        1|         DF|      NULL|           NULL|            BRASÍLIA|  1| 824|
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2| 559|
|      M|          3|        1|         ES|       416|            320|           CARIACICA|  3| 718|
|      M|          3|        1|         PA|      NULL|           NULL|         SALINÓPOLIS|  4| 647|
|      F|          3|        1|         RJ|      NULL|           NULL|      RIO DE JANEIRO|  5| 799|
|      M|          2|        1|         PE|      NULL|           NULL|              RECIFE|

In [19]:
### Contagem de registros do Dataframe Enem antes da exlusão dos registros sem nota de Matemática ou Redação
df.count()

1020883

In [20]:
### Exclusão dos registros nulos de notas de Matemática e Redação
df = df.na.drop(how="any")

In [21]:
df.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id|IDH|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2|559|
|      M|          3|        1|         ES|       416|            320|           CARIACICA|  3|718|
|      F|          2|        1|         RJ|       405|            440|         SÃO GONÇALO|  7|739|
|      F|          3|        1|         PE|       435|            360|           ARCOVERDE|  9|667|
|      F|          1|        3|         SE|       457|            940|NOSSA SENHORA DA ...| 11|587|
|      F|          3|        1|         SP|       536|            640|       CARAGUATATUBA| 17|759|
|      F|          3|        1|         PE|       463|              0|             CARPINA| 20|680|


In [22]:
### Nova contagem do Dataframe
df.count()

668630

In [23]:
### Visualização dos tipos de dados
df.printSchema()

root
 |-- TP_SEXO: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- SG_UF_PROVA: string (nullable = true)
 |-- NU_NOTA_MT: string (nullable = true)
 |-- NU_NOTA_REDACAO: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- id: long (nullable = false)
 |-- IDH: string (nullable = true)



In [24]:
### Correção do tipo de dados para campos quantitativos
df = df.withColumn("NU_NOTA_MT",df.NU_NOTA_MT.cast('int'))
df = df.withColumn("NU_NOTA_REDACAO",df.NU_NOTA_REDACAO.cast('int'))
df = df.withColumn("IDH",df.IDH.cast('int'))

In [25]:
df.printSchema()

root
 |-- TP_SEXO: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- SG_UF_PROVA: string (nullable = true)
 |-- NU_NOTA_MT: integer (nullable = true)
 |-- NU_NOTA_REDACAO: integer (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- id: long (nullable = false)
 |-- IDH: integer (nullable = true)



## **Análise de Dados**

In [26]:
### Visualizar a nota de Matemática e Redação por ID
df.select("id", "NU_NOTA_REDACAO", "NU_NOTA_MT").show()

+---+---------------+----------+
| id|NU_NOTA_REDACAO|NU_NOTA_MT|
+---+---------------+----------+
|  2|            760|       565|
|  3|            320|       416|
|  7|            440|       405|
|  9|            360|       435|
| 11|            940|       457|
| 17|            640|       536|
| 20|              0|       463|
| 22|            640|       343|
| 23|            760|       625|
| 25|            520|       383|
| 27|            640|       573|
| 29|            300|       615|
| 30|            640|       745|
| 31|            360|       352|
| 33|            760|       536|
| 36|            540|       446|
| 37|            360|       475|
| 38|            420|       409|
| 39|            460|       568|
| 40|            560|       430|
+---+---------------+----------+
only showing top 20 rows



In [27]:
 ### Verificar a média das notas de Redação por Estado
(
    df
    .groupby("SG_UF_PROVA")
    .agg({'NU_NOTA_REDACAO': 'avg'})
    .show()
)

+-----------+--------------------+
|SG_UF_PROVA|avg(NU_NOTA_REDACAO)|
+-----------+--------------------+
|         SC|   656.6800672465463|
|         RO|   616.2198952879581|
|         PI|   625.6701692072419|
|         AM|   557.5930632381522|
|         RR|   589.8901098901099|
|         GO|   635.0904704463209|
|         TO|   621.2470952749807|
|         MT|   616.5049258815947|
|         SP|   653.5130295363921|
|         ES|   658.8003132341425|
|         PB|   632.1715463917526|
|         RS|   649.0904033379694|
|         MS|   613.6243781094528|
|         AL|   620.9066940090815|
|         MG|   659.7096317280453|
|         PA|   605.1226056803171|
|         BA|   618.4151148132191|
|         SE|   642.5996015936255|
|         PE|   627.5359668960208|
|         CE|   600.9539911394982|
+-----------+--------------------+
only showing top 20 rows



In [28]:
 ### Verificar a média das notas de Matemática por Estado
(
    df
    .groupby("SG_UF_PROVA")
    .agg({'NU_NOTA_MT': 'avg'})
    .show()
)

+-----------+------------------+
|SG_UF_PROVA|   avg(NU_NOTA_MT)|
+-----------+------------------+
|         SC| 562.7392003508515|
|         RO| 514.2657940663177|
|         PI| 513.9361649647279|
|         AM| 489.7610255643594|
|         RR|511.21217244294166|
|         GO| 530.1149861633435|
|         TO| 514.7424477149497|
|         MT| 523.5862259460455|
|         SP| 564.2274208493404|
|         ES| 553.8880187940485|
|         PB| 517.3012371134021|
|         RS| 550.8450625869262|
|         MS| 524.2993781094527|
|         AL| 516.6628094331331|
|         MG| 559.0131019830028|
|         PA| 493.1115422721268|
|         BA| 510.1659884787847|
|         SE| 513.0799925298804|
|         PE| 529.6026079006881|
|         CE| 521.2425992892813|
+-----------+------------------+
only showing top 20 rows



In [29]:
### Verificar as notas mais altas de Redação por Estado
(
    df
    .groupby("SG_UF_PROVA")
    .agg({'NU_NOTA_REDACAO': 'max'})
    .show()
)

+-----------+--------------------+
|SG_UF_PROVA|max(NU_NOTA_REDACAO)|
+-----------+--------------------+
|         SC|                 980|
|         RO|                 980|
|         PI|                 980|
|         AM|                 980|
|         RR|                 980|
|         GO|                 980|
|         TO|                 980|
|         MT|                 980|
|         SP|                 980|
|         ES|                 980|
|         PB|                1000|
|         RS|                 980|
|         MS|                 980|
|         AL|                 980|
|         MG|                 980|
|         PA|                 980|
|         BA|                 980|
|         SE|                 980|
|         PE|                 980|
|         CE|                 980|
+-----------+--------------------+
only showing top 20 rows



In [30]:
### Verificar a média das notas agrupadas por Estado e Gênero
(
    df
    .groupby("SG_UF_PROVA", "TP_SEXO")
    .agg({'NU_NOTA_REDACAO': 'avg'})
    .show()
)

+-----------+-------+--------------------+
|SG_UF_PROVA|TP_SEXO|avg(NU_NOTA_REDACAO)|
+-----------+-------+--------------------+
|         SC|      M|   635.7577319587629|
|         RS|      M|   630.9337387558596|
|         AC|      M|   599.2667794698251|
|         SC|      F|   664.9627589021528|
|         RR|      F|    598.021712907117|
|         SP|      F|   661.7404686109141|
|         MA|      F|   612.2147383126279|
|         SE|      M|   630.4562178072111|
|         MA|      M|   600.5927283837458|
|         RO|      F|   625.5755395683453|
|         RN|      F|   631.8371590818203|
|         CE|      M|   572.2705205765927|
|         PE|      M|   612.4181342386307|
|         MT|      F|   622.5043996209557|
|         AL|      F|   628.6634103019538|
|         AM|      M|   546.6888242625676|
|         RR|      M|   570.8474576271186|
|         PR|      F|   645.7289719626168|
|         PI|      F|   628.8918597742127|
|         SP|      M|    635.195162915687|
+----------

In [31]:
### Verificar a média das notas de Redação por Gênero para o Estado de Mato Grosso
(
    df
    .where(df.SG_UF_PROVA == 'MT')
    .groupby("SG_UF_PROVA", "TP_SEXO")
    .agg({'NU_NOTA_REDACAO': 'avg'})
    .show()
)

+-----------+-------+--------------------+
|SG_UF_PROVA|TP_SEXO|avg(NU_NOTA_REDACAO)|
+-----------+-------+--------------------+
|         MT|      F|   622.5043996209557|
|         MT|      M|    603.747841105354|
+-----------+-------+--------------------+



In [32]:
### Estatística Geral
(
    df
    .describe()
    .show()
)

+-------+-------+------------------+-----------------+-----------+-----------------+------------------+------------+-------------------+-----------------+
|summary|TP_SEXO|       TP_COR_RACA|        TP_ESCOLA|SG_UF_PROVA|       NU_NOTA_MT|   NU_NOTA_REDACAO|NO_MUNICIPIO|                 id|              IDH|
+-------+-------+------------------+-----------------+-----------+-----------------+------------------+------------+-------------------+-----------------+
|  count| 668630|            668630|           668630|     668630|           668630|            668630|      668630|             668630|           668630|
|   mean|   NULL|2.0911371012368574|1.514351734142949|       NULL|529.1915498855869| 628.3884061439062|        NULL|3.887366719379355E9|712.6342850305849|
| stddev|   NULL|1.0118603321914381|0.603364808912964|       NULL|112.5220634849323|204.14110463708386|        NULL|4.275537308465593E9|76.80470640723459|
|    min|      F|                 0|                1|         AC|    

## **Montagem do Dataframe para o Modelo**

In [33]:
df.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id|IDH|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2|559|
|      M|          3|        1|         ES|       416|            320|           CARIACICA|  3|718|
|      F|          2|        1|         RJ|       405|            440|         SÃO GONÇALO|  7|739|
|      F|          3|        1|         PE|       435|            360|           ARCOVERDE|  9|667|
|      F|          1|        3|         SE|       457|            940|NOSSA SENHORA DA ...| 11|587|
|      F|          3|        1|         SP|       536|            640|       CARAGUATATUBA| 17|759|
|      F|          3|        1|         PE|       463|              0|             CARPINA| 20|680|


In [34]:
### Execução dos 'String Indexer'
tp_sexo_index = StringIndexer(inputCol="TP_SEXO", outputCol="INDEXED_TP_SEXO")
model_tp_sexo = tp_sexo_index.fit(df)
df = model_tp_sexo.transform(df)

tp_cor_raca_index = StringIndexer(inputCol="TP_COR_RACA", outputCol="INDEXED_TP_COR_RACA")
model_tp_cor_raca = tp_cor_raca_index.fit(df)
df = model_tp_cor_raca.transform(df)

tp_escola_index = StringIndexer(inputCol="TP_ESCOLA", outputCol="INDEXED_TP_ESCOLA")
model_tp_escola = tp_escola_index.fit(df)
df = model_tp_escola.transform(df)

sg_uf_index = StringIndexer(inputCol="SG_UF_PROVA", outputCol="INDEXED_SG_UF_PROVA")
model_sg_uf = sg_uf_index.fit(df)
df = model_sg_uf.transform(df)

no_municipio_index = StringIndexer(inputCol="NO_MUNICIPIO", outputCol="INDEXED_NO_MUNICIPIO")
model_no_municipio = no_municipio_index.fit(df)
df = model_no_municipio.transform(df)

In [35]:
### Aplicação do 'Hot Encoder'
encoders = OneHotEncoder(
    inputCols=["INDEXED_TP_SEXO", "INDEXED_TP_COR_RACA", "INDEXED_TP_ESCOLA", "INDEXED_SG_UF_PROVA", "INDEXED_NO_MUNICIPIO"],
    outputCols=["TP_SEXO_VEC", "TP_COR_RACA_VEC", "TP_ESCOLA_VEC", "SG_UF_PROVA_VEC", "NO_MUNICIPIO_VEC"],
    dropLast=False
    )

In [36]:
df = encoders.fit(df).transform(df)
df.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+---------------+-------------------+-----------------+-------------------+--------------------+-------------+---------------+-------------+---------------+-------------------+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id|IDH|INDEXED_TP_SEXO|INDEXED_TP_COR_RACA|INDEXED_TP_ESCOLA|INDEXED_SG_UF_PROVA|INDEXED_NO_MUNICIPIO|  TP_SEXO_VEC|TP_COR_RACA_VEC|TP_ESCOLA_VEC|SG_UF_PROVA_VEC|   NO_MUNICIPIO_VEC|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+---------------+-------------------+-----------------+-------------------+--------------------+-------------+---------------+-------------+---------------+-------------------+
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2|559|            0.0|                2.0|              0.0|                2.0|  

## **Montando o vetor de features**

In [37]:
assembler = VectorAssembler(inputCols=["TP_SEXO_VEC", "TP_COR_RACA_VEC", "TP_ESCOLA_VEC", "SG_UF_PROVA_VEC", "NU_NOTA_MT", "NU_NOTA_REDACAO", "NO_MUNICIPIO_VEC", "IDH"], outputCol="features")
df_features = assembler.transform(df)

In [38]:
### Visualização do resultado das 'features'
df_features.select(df_features.features).show(20, False)

+-------------------------------------------------------------------------+
|features                                                                 |
+-------------------------------------------------------------------------+
|(1567,[0,4,8,13,38,39,967,1566],[1.0,1.0,1.0,1.0,565.0,760.0,1.0,559.0]) |
|(1567,[1,2,8,28,38,39,132,1566],[1.0,1.0,1.0,1.0,416.0,320.0,1.0,718.0]) |
|(1567,[0,4,8,14,38,39,66,1566],[1.0,1.0,1.0,1.0,405.0,440.0,1.0,739.0])  |
|(1567,[0,2,8,16,38,39,257,1566],[1.0,1.0,1.0,1.0,435.0,360.0,1.0,667.0]) |
|(1567,[0,3,10,24,38,39,280,1566],[1.0,1.0,1.0,1.0,457.0,940.0,1.0,587.0])|
|(1567,[0,2,8,11,38,39,596,1566],[1.0,1.0,1.0,1.0,536.0,640.0,1.0,759.0]) |
|(1567,[0,2,8,16,38,311,1566],[1.0,1.0,1.0,1.0,463.0,1.0,680.0])          |
|(1567,[0,3,8,14,38,39,380,1566],[1.0,1.0,1.0,1.0,343.0,640.0,1.0,715.0]) |
|(1567,[1,2,8,12,38,39,42,1566],[1.0,1.0,1.0,1.0,625.0,760.0,1.0,754.0])  |
|(1567,[1,2,8,13,38,39,1446,1566],[1.0,1.0,1.0,1.0,383.0,520.0,1.0,557.0])|
|(1567,[1,4,

## **Executar o Modelo Não Supervisionado (Cluster) K-Means**

In [75]:
kmeans = KMeans().setK(2)
modeloK = kmeans.fit(df_features)
df_agrupado = modeloK.transform(df_features)
df_agrupado.show(truncate=False)

+-------+-----------+---------+-----------+----------+---------------+-------------------------+---+---+---------------+-------------------+-----------------+-------------------+--------------------+-------------+---------------+-------------+---------------+-------------------+-------------------------------------------------------------------------+----------+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|NO_MUNICIPIO             |id |IDH|INDEXED_TP_SEXO|INDEXED_TP_COR_RACA|INDEXED_TP_ESCOLA|INDEXED_SG_UF_PROVA|INDEXED_NO_MUNICIPIO|TP_SEXO_VEC  |TP_COR_RACA_VEC|TP_ESCOLA_VEC|SG_UF_PROVA_VEC|NO_MUNICIPIO_VEC   |features                                                                 |prediction|
+-------+-----------+---------+-----------+----------+---------------+-------------------------+---+---+---------------+-------------------+-----------------+-------------------+--------------------+-------------+---------------+-------------+---------------+-----------

**Avaliando o Modelo**

Faixa de valor da silhueta

S (i) ficará entre [-1, 1]

- Se o valor da silhueta for próximo a 1, a amostra está bem agrupada e já atribuída a um agrupamento muito apropriado.
- Se o valor da silhueta for cerca de 0, a amostra pode ser atribuída a outro cluster mais próximo a ele e a amostra fica igualmente longe de ambos os clusters. Isso significa que indica clusters sobrepostos
- Se o valor da silhueta estiver próximo a -1, a amostra foi classificada incorretamente e foi simplesmente colocada em algum lugar entre os clusters.

In [76]:
### 02 Clusters
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(df_agrupado)
print("Silhouette (02 clusters) com distância euclidiana quadrada = " + str(silhouette))

Silhouette (02 clusters) com distância euclidiana quadrada = 0.5362025135669117


In [77]:
### 03 Clusters
kmeans = KMeans().setK(3)
modeloK = kmeans.fit(df_features)
df_agrupado = modeloK.transform(df_features)
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(df_agrupado)
print("Silhouette (03 clusters) com distância euclidiana quadrada = " + str(silhouette))

Silhouette (03 clusters) com distância euclidiana quadrada = 0.5835498141963934


In [78]:
### 04 Clusters
kmeans = KMeans().setK(4)
modeloK = kmeans.fit(df_features)
df_agrupado = modeloK.transform(df_features)
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(df_agrupado)
print("Silhouette (04 clusters) com distância euclidiana quadrada = " + str(silhouette))

Silhouette (04 clusters) com distância euclidiana quadrada = 0.44795353146406425


In [79]:
### 05 Clusters
kmeans = KMeans().setK(5)
modeloK = kmeans.fit(df_features)
df_agrupado = modeloK.transform(df_features)
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(df_agrupado)
print("Silhouette (05 clusters) com distância euclidiana quadrada = " + str(silhouette))

Silhouette (05 clusters) com distância euclidiana quadrada = 0.4541263496398444
